In [1]:
# Data processing
import pandas as pd
import numpy as np
import datetime as dt

# Geographic data
import geopy
from geopy.geocoders import Nominatim
from geopy.distance import geodesic as geodesic
from mpl_toolkits.basemap import Basemap

# Visualization
import seaborn as sb
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

KeyError: 'PROJ_LIB'

In [2]:
import os

# Control Variables

In [ ]:
# PATH = "./all/basic_filter-"
PATH = "./all/basic_filter_drop_datetime"

# read whole file use None
# ROWS = None 
ROWS = 500_000

LEFT_BOTTOM_CITY = 'Tennessee'
RIGHT_TOP_CITY = 'Maine'

NYC_LONGITUDE = (-74.256436, -73.699733)
NYC_LATITUDE = (40.495029, 40.915592)

JFK_LONGITUDE = (-73.789185, -73.775291)
JFK_LATITUDE = (40.641960, 40.649562)

# Taxicab Rate of Fare
MILEAGE = 0.4 * 5

In [ ]:
# Read feather

# basic_filter_df = pd.read_feather(PATH+str(ROWS))
basic_filter_df = pd.read_feather(PATH)

In [ ]:
geolocator = Nominatim(user_agent='city')
LEFT_BOTTOM_BOUNDARY = geolocator.geocode(LEFT_BOTTOM_CITY)
RIGHT_TOP_BOUNDARY = geolocator.geocode(RIGHT_TOP_CITY)

# Auxiliary Functions

In [ ]:
def basic_pearson_corr_result(df, title):
    print('{title}, Pearson correlation coef. of:'.format(title=title))
    format_rule = '    {0:30}'
    # Euclidean distance of the ride and the taxi fare
    print(format_rule.format('dist and fare: '), \
          df['dist'].corr(df['fare_amount']))
    # time of day and distance traveled
    print(format_rule.format('time of day and dist: '), \
          df['hour'].corr(df['dist']))
    # time of day and the taxi fare
    print(format_rule.format('time of day and fare: '), \
          df['hour'].corr(df['fare_amount']))
    
def basic_visualization_result(df):
    df.plot(kind='scatter', x='dist', y='fare_amount')
    df.plot(kind='scatter', x='dist', y='hour')
    df.plot(kind='scatter', x='hour', y='fare_amount')
    
def compare_two_df(df1, title1, df2, title2):
    title_rule = '    {}'
    data_rule = '        {0:30}'

    # Euclidean distance of the ride and the taxi fare
    print('Pearson correlation coef. of:')
    print(title_rule.format('* dist and fare: '))
    print(data_rule.format(title1), \
          df1['dist'].corr(df1['fare_amount']))
    print(data_rule.format(title2), \
          df2['dist'].corr(df2['fare_amount']))

    print(title_rule.format('* time of day and dist: '))
    print(data_rule.format(title1), \
          df1['hour'].corr(df1['dist']))
    print(data_rule.format(title2), \
          df2['hour'].corr(df2['dist']))

    print(title_rule.format('* time of day and fare: '))
    print(data_rule.format(title1), \
          df1['hour'].corr(df1['fare_amount']))
    print(data_rule.format(title2), \
          df2['hour'].corr(df2['fare_amount']))

#     fig = plt.figure(figsize=(32,32))
#     ax1_1 = fig.add_subplot(321)
#     ax2_1 = fig.add_subplot(322)
#     ax1_2 = fig.add_subplot(323)
#     ax2_2 = fig.add_subplot(324)
#     ax1_3 = fig.add_subplot(325)
#     ax2_3 = fig.add_subplot(326)

#     df1.plot(ax=ax1_1, kind='scatter', x='dist', y='fare_amount', title = title1)
#     df1.plot(ax=ax1_2, kind='scatter', x='dist', y='hour', title = title1)
#     df1.plot(ax=ax1_3, kind='scatter', x='hour', y='fare_amount', title = title1)
#     df2.plot(ax=ax2_1, kind='scatter', x='dist', y='fare_amount', title = title2)
#     df2.plot(ax=ax2_2, kind='scatter', x='dist', y='hour', title = title2)
#     df2.plot(ax=ax2_3, kind='scatter', x='hour', y='fare_amount', title = title2)

## Basic Filter

In [ ]:
def valid_fare(row_data):
    """
    this is a soft cut based on initial chare $2.50 and the improvement surcharge 30-cent
    """
    return row_data['fare_amount'] > 2.8

def valid_transportation_region(row_data, left_bottom = LEFT_BOTTOM_BOUNDARY, right_top = RIGHT_TOP_BOUNDARY):
    """
    default region based on the rectangle formed by Tennessee and Maine, 
    since it takes more than 12 hrs to drive from the boundary to NYC or
    from NYC to the boundary
    """
    return (row_data['pickup_longitude'] > left_bottom.longitude) & \
           (row_data['pickup_longitude'] < right_top.longitude) & \
           (row_data['dropoff_longitude'] > left_bottom.longitude) & \
           (row_data['dropoff_longitude'] < right_top.longitude) & \
            (row_data['pickup_latitude'] > left_bottom.latitude) & \
            (row_data['pickup_latitude'] < right_top.latitude) & \
            (row_data['dropoff_latitude'] > left_bottom.latitude) & \
            (row_data['dropoff_latitude'] < right_top.latitude)

def valid_travel_location(row_data):
    """
    pickup and dropoff should not be exactly the same or
    both outside the NYC 
    """
    return (row_data['pickup_longitude'] != row_data['dropoff_longitude']) | \
           (row_data['pickup_latitude'] != row_data['dropoff_latitude']) | \
           (\
            (row_data['pickup_longitude'] >= NYC_LONGITUDE[0]) & \
            (row_data['pickup_longitude'] <= NYC_LONGITUDE[1]) & \
            (row_data['pickup_latitude'] >= NYC_LATITUDE[0]) & \
            (row_data['pickup_latitude'] <= NYC_LATITUDE[1]) \
           ) | (\
            (row_data['dropoff_longitude'] >= NYC_LONGITUDE[0]) & \
            (row_data['dropoff_longitude'] <= NYC_LONGITUDE[1]) & \
            (row_data['dropoff_latitude'] >= NYC_LATITUDE[0]) & \
            (row_data['dropoff_latitude'] <= NYC_LATITUDE[1]) \
           )

def valid_passenger_count(row_data):
    """
    by Taxi rule, the maximum capacity is 6
    """
    return (row_data['passenger_count'] > 0) & (row_data['passenger_count'] <= 6)

# def datetime_parser(x):
#     print(x['pickup_datetime'])
#     try:
#         datetime_object = datetime.strptime(x['pickup_datetime'], '%Y-%m-%d %H:%M:%S')
#         return datetime_object.hours*60 + datetime_object.minutes
#     except:
#         return pd.NaT

## Advanced Filter

In [ ]:
def valid_fare_amount_judge_by_travel_distance(row_data):
    return valid_fare_amount_with_long_distance(row_data) & \
           valid_fare_amount_with_short_distance(row_data)

def valid_fare_amount_with_long_distance(row_data):
    return row_data['fare_amount'] >= (row_data['dist'] * MILEAGE + 2.8)

def valid_fare_amount_with_short_distance(row_data):
    return (row_data['dist'] >= 0.2) | (row_data['fare_amount'] <= 10)

def valid_fare_with_upperbound(row_data, amount = 500):
    return row_data['fare_amount'] < amount
    
def to_jfk(row_data):
    if ((row_data['dropoff_longitude'] >= JFK_LONGITUDE[0]) & \
       (row_data['dropoff_longitude'] <= JFK_LONGITUDE[1]) & \
       (row_data['dropoff_latitude'] >= JFK_LATITUDE[0]) & \
       (row_data['dropoff_latitude'] <= JFK_LATITUDE[1])):
        return 1
    else:
        return 0
    
def from_jfk(row_data):
    if ((row_data['pickup_longitude'] >= JFK_LONGITUDE[0]) & \
        (row_data['pickup_longitude'] <= JFK_LONGITUDE[1]) & \
        (row_data['pickup_latitude'] >= JFK_LATITUDE[0]) & \
        (row_data['pickup_latitude'] <= JFK_LATITUDE[1])):
        return 1
    else:
        return 0

def remove_outliers_Tukey(usefulData, attr):
    thirdQuartile = usefulData.quantile(.75)[attr]
    firstQuartile = usefulData.quantile(.25)[attr]
    IQR = thirdQuartile - firstQuartile
    return usefulData[usefulData[attr].between(firstQuartile - (IQR * 1.5), thirdQuartile + (IQR * 1.5))]

# Basic Visualization

In [ ]:
lat = basic_filter_df.pickup_latitude.values
lon = basic_filter_df.pickup_longitude.values
# population = cities['population_total'].values
# area = cities['area_total_km2'].values

NYC_LONGITUDE = (-74.256436, -73.699733)
NYC_LATITUDE = (40.495029, 40.915592)

m = Basemap(projection='merc', lat_0 = 40.5, lon_0 = -75, resolution = 'h', area_thresh = 0.1,
    llcrnrlon=-80, llcrnrlat=35,
    urcrnrlon=-65, urcrnrlat=45)

m.drawmapboundary(fill_color='aqua')
m.fillcontinents(color='#ddaa66',lake_color='aqua')
m.drawcoastlines()

x,y = m(lon, lat)

m.plot(x, y, 'ro', markersize=1)

m.shadedrelief()

plt.show()

# https://peak5390.wordpress.com/2012/12/08/matplotlib-basemap-tutorial-plotting-points-on-a-simple-map/

## Adjust or Create Feature

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def dist(row_data):
    R = 6373.0
    s_lon = radians(row_data['pickup_longitude'])
    s_lat = radians(row_data['pickup_latitude'])
    e_lon = radians(row_data['dropoff_longitude'])
    e_lat = radians(row_data['dropoff_latitude'])
    diff_lon = e_lon - s_lon
    diff_lat = e_lat - s_lat

    a = sin(diff_lat / 2)**2 + cos(s_lat) * cos(e_lat) * sin(diff_lon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 0.621371

def dist_by_geopy(row_data):
    s_lon = row_data['pickup_longitude']
    s_lat = row_data['pickup_latitude']
    e_lon = row_data['dropoff_longitude']
    e_lat = row_data['dropoff_latitude']
    return geodesic((s_lon, s_lat), (e_lon, e_lat)).miles

# clean_data['dist'] = np.vectorize(dist)(clean_data['pickup_longitude'], clean_data['pickup_latitude'], clean_data['dropoff_longitude'], clean_data['dropoff_latitude'])
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/19412565

def insert_est_or_edt_time(df):
    df.insert(df.shape[1],'new_york_time',
              df.pickup_datetime.dt.tz_localize('utc').\
              dt.tz_convert('America/New_York'))
    
def insert_year(df):
    df.insert(df.shape[1], 'year', df.new_york_time.dt.year)

def insert_weekday(df):
    df.insert(df.shape[1], 'weekday', df.new_york_time.dt.weekday)

def insert_time_of_day(df):
    df.insert(df.shape[1], 'hour', df.new_york_time.dt.hour + df.new_york_time.dt.minute/60)

# def insert_dist(df, func = dist):
#     euclidean_dist = df.apply(func, axis = 1)
#     df.insert(df.shape[1], 'dist', euclidean_dist)
    
def insert_feature(df, func = None, feature_name = None):
    new_data = df.apply(func, axis = 1)
    df.insert(df.shape[1], feature_name, new_data)

# LinearRegression

In [ ]:
def simple_linear_regression(df, test_size = 0.3):
    train, test = train_test_split(df, test_size = test_size)
    train_x = list(map(lambda x: [x], list(train.dist)))
    train_y = list(map(lambda x: [x], list(train.fare_amount)))
    test_x = list(map(lambda x: [x], list(test.dist)))
    test_y = list(map(lambda x: [x], list(test.fare_amount)))

    regr = linear_model.LinearRegression()
    regr.fit(train_x, train_y)
    fare_predict = regr.predict(test_x)

    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print("Mean squared error: %.2f"
          % mean_squared_error(test_y, fare_predict))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(test_y, fare_predict))

    # Plot outputs
    plt.scatter(test_x, test_y,  color='blue')
    plt.plot(test_x, fare_predict, color='red', linewidth=3)

    plt.xticks(())
    plt.yticks(())

    plt.show()

In [ ]:
def adjust_fare(row_data):
    fare = row_data['fare_amount']
    
    if to_jfk(row_data): # should consider from_jfk too 
        # flat fare of $52
        fare -= 52
        # MTA State Surcharge
        fare -= 0.5
        # 30-cent Improvement Surcharge,
        fare -= 0.3
        
        if is_rush(row_data):
            fare -= 4.5
    else:
        pass

# Q1 Take a look at the training data ...

## There may be anomalies in the data that you may need to factor in before you start on the other tasks. Clean the data first to handle these issues. Explain what you did to clean the data (in bulleted form).

# Data Cleaning and Visualization
---

## Pearson Correlation Coefficients, 
## and Visualization of the Relation between the Variables

In [ ]:
advanced_df = basic_filter_df.loc[valid_fare_amount_judge_by_travel_distance(basic_filter_df)]
advanced_w_upperbound = advanced_df[valid_fare_with_upperbound(advanced_df, 300)]

compare_two_df(basic_filter_df, 'Applied basic rules', advanced_df, 'Applied travel distance rules')
compare_two_df(advanced_df, 'Applied travel distance rules', advanced_w_upperbound, 'Applied upperbound fare rule')

In [ ]:
fig, ax = plt.subplots()
basic_filter_df.fare_amount.hist(ax = ax, bottom=0.01)
ax.set_yscale('log')

In [ ]:
fig, ax = plt.subplots()
advanced_w_upperbound.fare_amount.hist(ax = ax, bottom=0.01)
ax.set_yscale('log')

In [ ]:
basic_filter_df.loc[basic_filter_df.fare_amount > 300]

In [ ]:
base_df = advanced_w_upperbound[:]

In [ ]:
def plot_hist(series, bins = 5, log = True):
    fig, ax = plt.subplots()
    series.hist(bins = bins, ax = ax, bottom=0.0001)
    if log:
        ax.set_yscale('log')
    ticks = [val for val in range(bins)]
    plt.setp(ax, xticks=ticks)

# Use the pyplot interface to change just one subplot...

    fig.tight_layout()
    plt.show()

In [ ]:
def is_weekday(row_data):
    return row_data['weekday'] < 5

# Histogram

---

## Weekday

---

In [ ]:
weekdays_df = base_df.loc[weekdays(base_df)]

In [ ]:
fig, ax = plt.subplots()
weekdays_df.fare_amount.hist(ax = ax, bottom=0.01)
ax.set_yscale('log')
# weekdays_df.weekday.hist(bins=5, grid = False)

In [ ]:
fig = plt.figure()
plt.hist(weekdays_df.weekday, bins=5)
plt.xticks(range(5))
plt.yscale('log')
plt.show()

## Weekend

In [ ]:
weekends_df = base_df.loc[~weekdays(base_df)]

In [ ]:
weekends_df.hist()

# Create New Feature
---

In [ ]:
insert_feature(base_df, func = to_jfk, feature_name = 'to_jfk')
insert_feature(base_df, func = from_jfk, feature_name = 'from_jfk')

In [ ]:
to_from_jfk_res = base_df.apply(to_from_jfk, axis = 1)

In [ ]:
base_df.describe()

In [ ]:
# compare_two_df(base_df, 'base case', weekdays_df, 'weekday only')
# compare_two_df(base_df, 'base case', weekends_df, 'weekend only')
# compare_two_df(base_df, 'base case', jfk_df, 'jfk only')

In [ ]:
insert_feature(base_df, func = dist, feature_name = 'dist2')

In [ ]:
base_df.describe()

In [ ]:
# simple_linear_regression(advanced_filter_df)
to_or_from_jfk_df = base_df.loc[(base_df['to_jfk'] == 1) | (base_df['from_jfk'] == 1)]
to_jfk_df = base_df.loc[(base_df['to_jfk'] == 1) & (base_df['from_jfk'] == 0)]
from_jfk_df = base_df.loc[(base_df['to_jfk'] == 0) & (base_df['from_jfk'] == 1)]
not_in_jfk_df = base_df.loc[(base_df['to_jfk'] == 0) & (base_df['from_jfk'] == 0)]

In [ ]:
# simple_linear_regression(advanced_w_upperbound)

# From Anywhere to JFK
* assume anywhere $\in$ Manhattan

---

# From JFK to Anywhere

* assume anywhere $\in$ Manhattan

---

In [ ]:
# basic_pearson_corr_result(advanced_filter_df, 'Applied advanced filter')
# basic_visualization_result(advanced_filter_df)

In [ ]:
# advanced_w_upperbound.describe()

In [ ]:
# advanced_df.describe()

# Simple Linear Regression Model

# Prediction Baseline

In [ ]:
# test_df_dist = test_df.apply(dist, axis=1)
# test_df.insert(test_df.shape[1], 'dist', test_df_dist)

In [ ]:
# test_X = list(map(lambda x: [x], list(test_df.dist)))

In [ ]:
# fare_predict = regr.predict(test_X).round(decimals = 2)

# Create submission file

In [ ]:
# submission = pd.DataFrame({'key': test_df.key,\
#                            'fare_amount': fare_predict.ravel()},\
#                           columns = ['key', 'fare_amount'])
# submission.to_csv('submission.csv', index = False)


In [ ]:
# submission.shape

# Extra Features 
## By year
## By weekday
## By time_of_day (hour)

## Extra Features - Adjusted_Fare_Amount_By_Rules

In [ ]:
# headers = list(advanced_filter_df.columns.values)

# for header in headers:
#     print(header)

In [ ]:
# headers = zip(list(advanced_filter_df.columns.values), list(base_filter_df.columns.values))

In [ ]:
# for h1, h2 in headers:
#     print('{:30}, {:30}'.format(h1, h2))

In [ ]:
# years_df = [None for _ in range(min(clean_data.year), max(clean_data.year) + 1)]
# minimum_year = min(clean_data.year)

# years = max(clean_data.year) + 1 - minimum_year
# for year in range(years):
#     years_df[year] = clean_data[clean_data.year == minimum_year+year]
      
#     train, test = train_test_split(years_df[year], test_size=0.2)
#     train_x = list(map(lambda x: [x], list(train.dist)))
#     train_y = list(map(lambda x: [x], list(train.fare_amount)))
#     test_x = list(map(lambda x: [x], list(test.dist)))
#     test_y = list(map(lambda x: [x], list(test.fare_amount)))
#     regr.fit(train_x, train_y)
#     fare_predict = regr.predict(test_x)

#     print(years_df[year]['dist'].corr(years_df[year]['fare_amount']))
#     # The coefficients
#     print('Year: {}'.format(minimum_year + year), 'Coeff: ', regr.coef_, \
#           "M.S.E: %.2f" % (mean_squared_error(test_y, fare_predict)), \
#           'Variance score: %.2f' % r2_score(test_y, fare_predict))
#     print('\n')

    # Plot outputs
#     plt.scatter(test_x, test_y,  color='blue')
#     plt.plot(test_x, fare_predict, color='red', linewidth=3)

#     plt.xticks(())
#     plt.yticks(())

    
# plt.show()

# Night Surcharge

In [ ]:
# def night_surcharge(row_data, night = True):
#     if night:
#         return (row_data['hour'] >= 20) | (row_data['hour'] <= 6)
#     else:
#         return (row_data['hour'] < 20) & (row_data['hour'] > 6)
    
# def weekday_filter(row_data):
#     return row_data.weekday < 5 


In [ ]:
# for year in range(years):
#     night = None
#     night = years_df[year][night_surcharge(years_df[year], True)]
#     train, test = train_test_split(night, test_size=0.2)
#     train_x = list(map(lambda x: [x], list(train.dist)))
#     train_y = list(map(lambda x: [x], list(train.fare_amount)))
#     test_x = list(map(lambda x: [x], list(test.dist)))
#     test_y = list(map(lambda x: [x], list(test.fare_amount)))
#     regr.fit(train_x, train_y)
#     fare_predict = regr.predict(test_x)

#     print(night['dist'].corr(night['fare_amount']))
#     # The coefficients
#     print('Year: {}'.format(minimum_year + year), 'Coeff: ', regr.coef_, \
#           "M.S.E: %.2f" % (mean_squared_error(test_y, fare_predict)), \
#           'Variance score: %.2f' % r2_score(test_y, fare_predict))
#     print('\n')

    # Plot outputs
#     plt.scatter(test_x, test_y,  color='blue')
#     plt.plot(test_x, fare_predict, color='red', linewidth=3)

#     plt.xticks(())
#     plt.yticks(())

    
# plt.show()

In [ ]:
# for year in range(years):
#     flag = [True, False]
#     weekday = None
#     weekday = years_df[year][weekday_filter(years_df[year])]
#     train, test = train_test_split(weekday, test_size=0.2)
#     train_x = list(map(lambda x: [x], list(train.dist)))
#     train_y = list(map(lambda x: [x], list(train.fare_amount)))
#     test_x = list(map(lambda x: [x], list(test.dist)))
#     test_y = list(map(lambda x: [x], list(test.fare_amount)))
#     regr.fit(train_x, train_y)
#     fare_predict = regr.predict(test_x)

#     print(weekday['dist'].corr(weekday['fare_amount']))
#     # The coefficients
#     print('Year: {}'.format(minimum_year + year), 'Coeff: ', regr.coef_, \
#           "M.S.E: %.2f" % (mean_squared_error(test_y, fare_predict)), \
#           'Variance score: %.2f' % r2_score(test_y, fare_predict))
#     print('\n')

#     # Plot outputs
#     plt.scatter(test_x, test_y,  color='blue')
#     plt.plot(test_x, fare_predict, color='red', linewidth=3)

#     plt.xticks(())
#     plt.yticks(())

    
#     plt.show()